# Oracle

Oracle is a pretty old term in QC, referring to a specific way to write function $f(x)$. Oracle obeys the reversibility principle by keeping the argument unchanged (see: [uncomputing](070%20IF%20statement.ipynb)), and involving argument in the result value:

![](https://i.stack.imgur.com/qbu6i.png)

The most trivial oracles are 1-bit check:

- $f(x) = x$:
  - if $x = 1$, then $y = y \oplus 1 = \neg y$. Do we know such an operator?
- $f(x) = 1 - x$

In [ ]:
from qiskit import QuantumCircuit, BasicAer, execute

oracle = QuantumCircuit(2, name="$U_f$")

## TODO : a trivial oracle f(x)=x

## TODO : a trivial oracle f(x)=1-x


oracle.draw('mpl')

In [ ]:
for i in range(4):
    v = [0, 0, 0, 0]
    v[i] = 1
    qc = QuantumCircuit(2)
    qc.initialize(v)
    qc.append(oracle, [0, 1])
    qc.measure_all()
    v2 = execute(qc, backend=BasicAer.get_backend('statevector_simulator')).result().get_statevector().real
    display(qc.draw('mpl'))
    print(f"{v} -> {v2}")

## Observation!

According to expected behavior, any oracle is a controlled $X$ gate, but with a complicated controlling state:

![](https://qiskit.org/textbook/ch-algorithms/images/grover_boolean_oracle.svg)

## Corollary!

It experience [Phase Kickback](105%20Phase%20Kickback.ipynb) the same way: if acillary $y$ qubit is in an eignstate of $X$ (those are $|-\rangle$ and $|+\rangle$), ONLY the satisfying ($f(x) = 1$) argument accepts a corresponding phase change. 

### MVE:

In [ ]:
qc = QuantumCircuit(3, 2)
# |00> + |01> + |10> + |11>
qc.initialize([0.5, 0.5, 0.5, 0.5], [0, 1])
v2 = execute(qc, backend=BasicAer.get_backend('statevector_simulator')).result().get_statevector().real
print(v2.round(3))
# |->, eigenstate
qc.initialize([.5 ** .5, -.5 ** .5], [2])
# oracle
qc.ccx(0, 1, 2)
qc.h(2)
qc.x(2)
v2 = execute(qc, backend=BasicAer.get_backend('statevector_simulator')).result().get_statevector().real
display(qc.draw('mpl'))
print(v2.round(3))

## Phase oracle...

... is an oracle, which is ready to be used for a phase kickback, i.e., ancilla is prepared in $|-\rangle$.

![](https://qiskit.org/textbook/ch-algorithms/images/grover_phase_oracle.svg)

# 2. Inversion about mean

In [ ]:
import matplotlib.pyplot as plt
array = [(1./32) ** .5] * 32

def f(x):
    """behavior of a phase oracle"""
    return -1 if (x == 17) else 1

def kickback(a):
    for i in range(len(a)):
        a[i] *= f(i)

In [ ]:
display(plt.bar(range(32), array))

In [ ]:
kickback(array)
display(plt.bar(range(32), array))

In [ ]:
def iom(a):
    avg = sum(a) / len(a)
    for i in range(len(a)):
        a[i] = avg - (a[i] - avg)

In [ ]:
iom(array)
display(plt.bar(range(32), array))

In [ ]:
# repeat multiple times

kickback(array)
iom(array)
display(plt.bar(range(32), array))